# Pandas
**Ian Allison, Compute Canada, 2020** (with several changes and additions for GOPH699)

Pandas (Python Data Analysis Library) is a swiss-army knife module that you'll find at the top of a huge proportion of notebooks. It's popular enough to deserve its own import idiom...

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/dtrad/geoml_course/blob/master/PandasIntroSolved.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
</table>

In [1]:
import numpy as np
import pandas as pd

To get a quick idea of the scope of pandas take a look at the autocomplete for pd.<TAB>

In [2]:
pd.api??

Type:        module
String form: <module 'pandas.api' from '/home/dtrad/anaconda3/envs/tfgpu4/lib/python3.9/site-packages/pandas/api/__init__.py'>
File:        ~/anaconda3/envs/tfgpu4/lib/python3.9/site-packages/pandas/api/__init__.py
Source:     
""" public toolkit API """
from pandas.api import (  # noqa
    extensions,
    indexers,
    types,
)


The fundamental objects in pandas are the `Series` and the `DataFrame`. Together they encapsulate how you will normally ingest, clean, manipulate and even visualize external data sources.  Most of this module will use methods on these objects so we'll take a quick tour of the concepts they implement, and the idea of an Index.

## Series

The pandas `Series` object is basically a one dimensional indexed array. Schematically, they look like

| Index | Value |
|-------|-------|
|   0   |  0.12 |
|   1   |  0.24 |
|   2   |  0.36 |
|   3   |  0.48 |


Where there are two columns: an index and a value. Most of the time the index values are distinct (not a firm requirement though!) but they don't have to be integers. Any hastable type will do.\
Def: a hash table (hash map) is a data structure that implements an associative array abstract data type, a structure that can map keys to values.


In [3]:
s1 = pd.Series({'one' : 1.0, 'two': 2.0, 'three': 3.0})
s1

one      1.0
two      2.0
three    3.0
dtype: float64

If not given index, it will create one by default:

The next thing to notice is that the values all have the same type (`dtype: float64` in this case). `pandas` can make `Series` of almost any type as long as all the rows share the same type, but it will try to pick the most efficient implementation (`numpy` `ndarrays` for numeric types). It can fall back to a generic `object` type if all else fails but it really pays (in speed) to keep an eye on the `dtype` and make it a `numpy` type if possible.

In [4]:
s2 = pd.Series(['one', 2, 'three'])
s2

0      one
1        2
2    three
dtype: object

One particularly common option is to use a timestamp as the index (don't worry about the syntax here, we'll come back to timeseries later)...

In [5]:
dti = pd.date_range('2020-06-03', periods=3, freq='H')
pd.Series(['first', 'second', 'third'], index=dti)

2020-06-03 00:00:00     first
2020-06-03 01:00:00    second
2020-06-03 02:00:00     third
Freq: H, dtype: object

### Making `Series`

You'll probably find that you are making `DataFrames` more often than `Series`, but lots of operations in `pandas` return Series or want them as an argument so it's good to know how to build them.

Like `numpy` `ndarray`s a sequence will work, it'll get the default ascending integer index

In [6]:
sf = pd.Series([1.0, 2.0, 3.0, 4.0, 5.0],dtype='float64')
sf

0    1.0
1    2.0
2    3.0
3    4.0
4    5.0
dtype: float64

In [7]:
si = pd.Series(np.arange(5))
si

0    0
1    1
2    2
3    3
4    4
dtype: int64

The `__repr__` includes the values we set along with the type of object we have stored (the values). As I mentioned, this is one of the attributes of a Series. \
There are other attributes like:

In [8]:
si.shape

(5,)

So far, this looks a lot like a numpy array (or even just a list), but we can switch the indexing to suit our needs, by explicitly passing the `index=` argument

In [9]:
sm = pd.Series([1., 2., 3., 4., 5.], index=['one', 'two', 'three', 'four', 'five'])
sm

one      1.0
two      2.0
three    3.0
four     4.0
five     5.0
dtype: float64

### Indexing and Slicing

You can access the values by keys (like a dictionary) or by attribute:

In [10]:
sm['three']

3.0

In [11]:
sm.three

3.0

Now the series is starting to look more like a dictionary, in fact, that's a pretty good way to construct series

In [12]:
sn = pd.Series({'one': 1, 'three': 3, 'two': 2, 'four': 4, 'five': 5})
sn

one      1
three    3
two      2
four     4
five     5
dtype: int64

In [13]:
sn.values

array([1, 3, 2, 4, 5])

In [14]:
sn.keys()

Index(['one', 'three', 'two', 'four', 'five'], dtype='object')

When you look closely though, a Series has a few tricks that a dictionary doesn't...

In [15]:
sn['three':'four']

three    3
two      2
four     4
dtype: int64

Label based indexes are _inclusive_ of the `stop` value. This is different from most other indexes you'll see in python, but this can cause a little confusion. Basically it boils down to the idea that there isn't always a natural "next" object in a hash (notice that the rows aren't sorted beyond what we specified manually in the index).

If you use numbers to do the slice, you'll get the familiar python behaviour


In [16]:
sn[1:3]

three    3
two      2
dtype: int64

Element by element statements evaluate to Booleans

In [17]:
sn > 2

one      False
three     True
two      False
four      True
five      True
dtype: bool

and if you remember the material on numpy fancy indexing, this can be very useful

In [18]:
sn[sn > 2]

three    3
four     4
five     5
dtype: int64

There are also some extra indexing methods available to you `.loc`, `.iloc`, `.ix`. Have a look at the documentation for these, they might seem redundant, but actually they're useful in some contexts where ordinary indexing will bite you...

In [19]:
sa = pd.Series(data=['apple', 'banana', 'orange', 'pineapple'], index=[3, 2, 1, 7])

sa

3        apple
2       banana
1       orange
7    pineapple
dtype: object

Suppose we want the value in the 4th row...

In [20]:
sa[3]

'apple'

Nope, `3` was interpreted as a label. There's an ambiguity because of the type of the index. Fortunately `pandas` gives us a pair of functions to avoid that problem

  * `.iloc[]`: Purely integer-location based indexing.
  * `loc[]` : Purely label based indexing.

In [21]:
sa.iloc[3]

'pineapple'

And `.iloc` will also work with slices

In [22]:
sa.iloc[:3]

3     apple
2    banana
1    orange
dtype: object

In the opposite situation

In [23]:
sa.loc[2]

'banana'

`iloc` does the opposite and tells pandas you want to use the implicit style python notation

In [24]:
sa.iloc[[2, 1, 3]]

1       orange
2       banana
7    pineapple
dtype: object

`ix` some combination of these things, but I've never been able to commit it to memory. Fortunately it is being deprecated in favour of `iloc` and `loc` so you don't need to worry 😀

## `DataFrames`

Most of the time you will be using `DataFrames` rather than `Series`, but at a first pass it is OK to think of `DataFrames` as a bunch of `Series` stuck together with a common index. 


| Index | Value1 | Value2 | 
|-------|--------|--------|
|   0   |  0.12  |   'a'  |
|   1   |  0.24  |   'b'  |
|   2   |  0.36  |   'c'  |
|   3   |  0.48  |   'd'  |

The rules discussed above for the index stay the same but notice that we can now have different types in the various columns

In [25]:
sn

one      1
three    3
two      2
four     4
five     5
dtype: int64

In [26]:
d1 = pd.DataFrame({'floats': sm, 'ints': sn})
d1

,floats,ints
five,5.0,5
four,4.0,4
one,1.0,1
three,3.0,3
two,2.0,2


### DataFrame Attributes

We've already seen some of the attributes of the DataFrame (column etc.) but there are quite a few available, take a look at `dtypes`, `ndim`, `shape`

In [27]:
d1.index

Index(['five', 'four', 'one', 'three', 'two'], dtype='object')

In [28]:
d1.columns

Index(['floats', 'ints'], dtype='object')

In [29]:
d1.shape

(5, 2)

In [30]:
d1.dtypes

floats    float64
ints        int64
dtype: object

### Indexing

When indexing a dataframe, the default is to give you the column (you can also use the syntax `d1.ints`)

In [31]:
d1['ints']


five     5
four     4
one      1
three    3
two      2
Name: ints, dtype: int64

If you are looking for the row, then try `.loc` with the row index value

In [32]:
d1.ints

five     5
four     4
one      1
three    3
two      2
Name: ints, dtype: int64

In [33]:
d1.loc['one']

floats    1.0
ints      1.0
Name: one, dtype: float64

As with series, you can give a list instead, but remember to count the parentheses...

In [34]:
d1.loc[['one','two']]

,floats,ints
one,1.0,1
two,2.0,2


Or you can use slices

In [35]:
d1.iloc[:2]

,floats,ints
five,5.0,5
four,4.0,4


Wherever possible, `pandas` (like `numpy`) will try to return a view on the same data rather than a copy, but because the indexing possibilities in `pandas` are much greater the specific rules are more subtle and we won't get into them now, suffice to say, if you need to break the link between two dataframes you can use the `.copy()` method

In [36]:
d1a = d1
d1a is d1

True

In [37]:
id(d1),id(d1a)

(140413125657600, 140413125657600)

In [38]:
d1b = d1.copy()
d1b is d1

False

In [39]:
id(d1),id(d1b)

(140413125657600, 140411336707520)

### Changing DataFrames

`DataFrames` are mutable; we can change the values in rows and columns and we can add/remove columns in place. `pandas` will usually try to do this in place, but some modifications (e.g. changing column dtype) require implicit data copies.

In [40]:
d1.iloc[1] = (3.0, 3)
d1

,floats,ints
five,5.0,5
four,3.0,3
one,1.0,1
three,3.0,3
two,2.0,2


Add a new column

In [41]:
d1['ratio'] = d1['ints'] / (2 * d1['floats'])
d1

,floats,ints,ratio
five,5.0,5,0.5
four,3.0,3,0.5
one,1.0,1,0.5
three,3.0,3,0.5
two,2.0,2,0.5


Sometimes not everything will fit in a single `DataFrame`, we'll see how to concatenate and otherwise join `DataFrames` later on.

## Filtering
Just like in the case of numpy, you can use an extra [] as a filter.

In [42]:
d1[d1.ints==3]

,floats,ints,ratio
four,3.0,3,0.5
three,3.0,3,0.5


## DataFrame Methods

There are *lots* of methods for operating on DataFrames, have a look at the tab completion and explore the documentation for them. In particular, take a look at the help for `describe`, `head` and `tail`. These are great for orienting yourself with a new dataset

In [43]:
d1.describe()

,floats,ints,ratio
count,5.00000,5.00000,5.0
mean,2.80000,2.80000,0.5
std,1.48324,1.48324,0.0
min,1.00000,1.00000,0.5
25%,2.00000,2.00000,0.5
50%,3.00000,3.00000,0.5
75%,3.00000,3.00000,0.5
max,5.00000,5.00000,0.5


We won't have time to dive into all of the methods but we'll sample a few, then you should explore the documentation for others. Here are some commonly used:

* `min`, `max`
* `mean`, `mode`, `median`
* `max`, `min`, `argmin`, `argmax`, `idxmax`, `idxmin`
* `any`, `all`
* `astype`
* `dropna`
* `shift`
* `sort_index`, `sort_values`

Generally these will return another `DataFrame` with the results you are looking for, but you can also pass the `inplace=True` keyword argument which will modify the `DataFrame` in place and save some memory.

In [44]:
d1.min()

floats    1.0
ints      1.0
ratio     0.5
dtype: float64

In [45]:
(d1 > 3).any()

floats     True
ints       True
ratio     False
dtype: bool

Think about that 1, `d1 > 3` returns a dataframe of booleans, any checks if there are any true values in each of the columns.

In [46]:
d1['floats'].astype(int)

five     5
four     3
one      1
three    3
two      2
Name: floats, dtype: int64

In [47]:
d1.sort_values('floats')

,floats,ints,ratio
one,1.0,1,0.5
two,2.0,2,0.5
four,3.0,3,0.5
three,3.0,3,0.5
five,5.0,5,0.5


`dropna` comes in VERY handy in combinations with other functions. For example, if your dataframe has a `NaN` in one row of a column, calling `.mean()` on that column won't work, but if you know that value can be safely ignored, you can do `.dropna().mean()`.

In [48]:
d1.head(2)

,floats,ints,ratio
five,5.0,5,0.5
four,3.0,3,0.5


### Hierarchical Indexing
This is an important feature in pandas that enables you to have multiple index levels on an axis, providing a way to access high dimensional data in a table format.

In [49]:
data = pd.Series(np.random.randn(9),index=[['a','a','a','b','b','c','c','d','d'],[1,2,3,1,3,1,2,2,3]])
data

a  1    0.808949
   2   -1.227848
   3    0.210808
b  1    0.373916
   3    1.166288
c  1    0.290373
   2    0.053918
d  2    0.075414
   3    2.493368
dtype: float64

Partial indexing enables us to select subsets of the data

In [50]:
data['b']

1    0.373916
3    1.166288
dtype: float64

We can access an inner level

In [51]:
data.loc[:,2]

a   -1.227848
c    0.053918
d    0.075414
dtype: float64

### Concatenate & Join

There are a handful of functions which handle concatenation. The main workhorse is `pd.concat`, but there are some convenience functions which will let you avoid passing lots of arguments to concat. You can combine either `Series` and `DataFrames` but we'll jump straight to `DataFrames`,

In [52]:
s1 = pd.DataFrame(
    {
        'upper': ['A', 'B', 'C'], 
        'lower': ['a', 'b', 'c']
    }, 
    columns=['upper', 'lower'], 
    index=[1,2,3]
)

s2 = pd.DataFrame(
    {
        'upper': ['D', 'E', 'F'],
        'lower': ['d', 'e', 'f']
    }, 
    columns=['upper', 'lower'],
    index=[4,5,6]
)

pd.concat([s1,s2])

,upper,lower
1,A,a
2,B,b
3,C,c
4,D,d
5,E,e
6,F,f


Notice that we enclosed the things we want to join as some sort of iterable (a `list` here).

What if we wanted to add columns rather than rows

In [53]:
t1 = pd.DataFrame({'upper': ['A', 'B', 'C'], 'lower': ['a', 'b', 'c']}, columns=['upper', 'lower'], index=[1,2,3])
t2 = pd.DataFrame({'greek': ['α', 'β', 'γ']}, index=[1,2,3])

pd.concat([t1, t2], axis='columns')

,upper,lower,greek
1,A,a,α
2,B,b,β
3,C,c,γ


`pd.concat` will accept duplicate indices, but normally that indicates a problem with the data normalization. `concat` has a `verify_index` argument which can check for these problems and you can specify what you want to do with duplicates manually.

concat will often result in `NaN`s because some columns might not exist in both/all frames.

In [54]:
d4 = pd.DataFrame({'fruit': ['apple', 'orange'], 'veg': ['brocolli', 'carrot'], 'tree': ['cedar', 'alder']})
d5 = pd.DataFrame({'veg': ['onion', 'potato'], 'fruit': ['banana','grape']})
pd.concat([d4,d5], sort=False, ignore_index=True)

,fruit,veg,tree
0,apple,brocolli,cedar
1,orange,carrot,alder
2,banana,onion,NaN
3,grape,potato,NaN


Since the index wasn't important here, I threw it away and just accepted a new one.

In the general case, joining DataFrames can get complex. The concat method can take a `join` keyword to specify a database like join strategy (inner or outer), but `pd.merge` is a bit more flexible. It implements the usual relations

  * one-to-one (similar to a concat)
  * many-to-one
  * many-to-many
  
For the many-to-one case here is an example

In [55]:
adf1=pd.DataFrame({
    'class': ['insect', 'spider'], 
    'legs': [6, 8]}
)

adf2=pd.DataFrame({
    'name': ['molly', 'anna', 'stephen', 'mica'], 
    'class': ['insect','insect','spider','insect']}
)                     

print(adf1); print(adf2)

    class  legs
0  insect     6
1  spider     8
      name   class
0    molly  insect
1     anna  insect
2  stephen  spider
3     mica  insect


In [56]:
pd.merge(adf1, adf2)

,class,legs,name
0,insect,6,molly
1,insect,6,anna
2,insect,6,mica
3,spider,8,stephen


The _many-to-one_ is many different rows in adf2 being mapped to a single row in adf1 (insects). `pd.merge` also accepts a selection of keyword arguments so you can manually specify which columns to join, patch up name differences etc.

## Working with External Data

Data comes in many, many forms from simple csv/json files, real-time APIs, structured binary files and many others. Try `pd.read_<TAB>` to see some of the `pandas` igestion options. `read_csv` is the main workhorse for data sets which will fit on a single machines. It is way more flexible than it's name suggests (S3 buckets, https, compressed files, ...) and many of the arguments to `read_csv` will have equivalents for the other functions, so we'll take a closer look at.
    
We need a CSV to work with. The city of Vancouver has an [open data catalog](https://vancouver.ca/your-government/open-data-catalogue.aspx), which has CSV for some of the datasets. There's a dataset which lists all of the community gardens and food trees maintained by the city. A copy of it is available in this directory called `CommunityGardensAndFoodTrees.csv 

In [57]:
pd.read_csv?

Signature:
pd.read_csv(
    filepath_or_buffer: 'FilePathOrBuffer',
    sep=<no_default>,
    delimiter=None,
    header='infer',
    names=<no_default>,
    index_col=None,
    usecols=None,
    squeeze=False,
    prefix=<no_default>,
    mangle_dupe_cols=True,
    dtype: 'DtypeArg | None' = None,
    engine=None,
    converters=None,
    true_values=None,
    false_values=None,
    skipinitialspace=False,
    skiprows=None,
    skipfooter=0,
    nrows=None,
    na_values=None,
    keep_default_na=True,
    na_filter=True,
    verbose=False,
    skip_blank_lines=True,
    parse_dates=False,
    infer_datetime_format=False,
    keep_date_col=False,
    date_parser=None,
    dayfirst=False,
    cache_dates=True,
    iterator=False,
    chunksize=None,
    compression='infer',
    thousands=None,
    decimal: 'str' = '.',
    lineterminator=None,
    quotechar='"',
    quoting=0,
    doublequote=True,
    escapechar=None,
    comment=None,
    encoding=None,
    encoding_errors: 'str | N

In [58]:
!tail CommunityGardensAndFoodTrees.csv

FA144;pre-1970;Valdez Park;3210;W;22nd;Av;3210 W 22nd Av, Vancouver, BC;0;6;;Apple;;Park Board;;;;Dunbar-Southlands;"{""type"": ""Point"", ""coordinates"": [-123.1764742, 49.2524296]}"
FA149;2013;Woodland Park Community Garden;705;;Woodland;Drive;705 Woodland Drive, Vancouver, BC;64;3;;Jonamac Apple, Italian Prune, Pacific Crabapple;;Park Board;Village Vancouver Transition Society;woodlandparkgarden@gmail.com;http://woodlandcommunitygarden.wordpress.com;Grandview-Woodland;"{""type"": ""Point"", ""coordinates"": [-123.07383824, 49.27845355]}"
FA156;2016;Pender Community Garden;2595;E;Pender;St;2595 E Pender St, Vancouver, BC;;;;;;Private;;;;Hastings-Sunrise;"{""type"": ""Point"", ""coordinates"": [-123.052293, 49.28068]}"
FA158;2010;Moberley Cultural Herb Garden;7646;;Prince Albert;St;7646 Prince Albert St, Vancouver, BC;5;;;;;Park Board;Moberly Arts and Cultural Centre;Moberlyarts@vancouver.ca;https://moberlyartscentre.ca/artists-residencies/;Sunset;"{""type"": ""Point"", ""coordinates

In [59]:
gardenDF = pd.read_csv(
    "CommunityGardensAndFoodTrees.csv",
    encoding='latin1',
    delimiter=';'
)
gardenDF.shape

(172, 19)

So there are 172 rows, with 19 columns, here are the fist few rows

In [60]:
gardenDF.head(10)

,MAPID,YEAR_CREATED,NAME,STREET_NUMBER,STREET_DIRECTION,STREET_NAME,STREET_TYPE,MERGED_ADDRESS,NUMBER_OF_PLOTS,NUMBER_OF_FOOD_TREES,NOTES,FOOD_TREE_VARIETIES,OTHER_FOOD_ASSETS,JURISDICTION,STEWARD_OR_MANAGING_ORGANIZATION,PUBLIC_E_MAIL,WEBSITE,Geo Local Area,Geom
0,FA004,2007,1755 West 14th - Private apartment building ga...,1755.0,W,14th,Av,"1755 W 14th Av, Vancouver, BC",NaN,NaN,NaN,NaN,NaN,Private,Apartment residents,NaN,NaN,Fairview,"{""type"": ""Point"", ""coordinates"": [-123.1448703..."
1,FA010,NaN,Barclay Heritage Square,1433.0,NaN,Barclay,St,"1433 Barclay St, Vancouver, BC",0.0,2,NaN,Apple,NaN,Park Board,NaN,NaN,NaN,West End,"{""type"": ""Point"", ""coordinates"": [-123.1328450..."
2,FA012,2011,Bosman Hotel,1060.0,NaN,Howe,St,"1060 Howe St, Vancouver, BC",12.0,NaN,NaN,NaN,NaN,Private,Bosman Hotel Society,NaN,NaN,Downtown,"{""type"": ""Point"", ""coordinates"": [-123.1250943..."
3,FA016,2013,Cambie Park Community Garden,500.0,W,54th,Av,"500 W 54th Av, Vancouver, BC",30.0,NaN,NaN,NaN,NaN,Park Board,Marpole Oakridge Community Association,southvangreenthumbs@gmail.com,http://cambieparkcommunitygarden.blogspot.com/,Oakridge,"{""type"": ""Point"", ""coordinates"": [-123.1186284..."
4,FA018,2011,Cartier Park,390.0,E,36th,Av,"390 E 36th Av, Vancouver, BC",0.0,6,NaN,Apple,NaN,Park Board,NaN,NaN,NaN,Riley Park,"{""type"": ""Point"", ""coordinates"": [-123.0975115..."
5,FA019,2008,Cedar Cottage Community Garden,2021.0,NaN,Stainsbury,Av,"2021 StainsburyÂ Av, Vancouver, BC",61.0,13,NaN,EARLY MCINTOSH; GOLDEN DELICIOUS SMOOTHEE APPL...,NaN,Translink,Cedar Cottage Garden Society,themembershipteam@cedarcottagegarden.org,http://www.cedarcottagegarden.org,Kensington-Cedar Cottage,"{""type"": ""Point"", ""coordinates"": [-123.0642552..."
6,FA020,2009,Cedar Cottage Community Garden - Part 2,2019.0,NaN,Stainsbury,Av,"2019 StainsburyÂ Av, Vancouver, BC",49.0,NaN,NaN,NaN,NaN,City,Cedar Cottage Garden Society,themembershipteam@cedarcottagegarden.org,http://www.cedarcottagegarden.org,Kensington-Cedar Cottage,"{""type"": ""Point"", ""coordinates"": [-123.0644448..."
7,FA021,2013,Cedar Cottage Neighourhood House Edible Garden,4065.0,NaN,Victoria,Drive,"4065 Victoria Drive, Vancouver, BC",12.0,NaN,NaN,NaN,NaN,Private,Cedar Cottage Neighbourhood House,NaN,NaN,Kensington-Cedar Cottage,"{""type"": ""Point"", ""coordinates"": [-123.0651476..."
8,FA022,NaN,Champlain Heights Park,3351.0,NaN,Maquinna,Drive,"3351 MaquinnaÂ Drive, Vancouver, BC",0.0,1,NaN,Apple,NaN,Park Board,NaN,NaN,NaN,Killarney,"{""type"": ""Point"", ""coordinates"": [-123.0326603..."
9,FA023,2010,Champlain Place Community Garden,3347.0,E,58th,Av,"3347 E 58th Av, Vancouver, BC",8.0,NaN,NaN,NaN,NaN,Private,BC Housing,NaN,NaN,Killarney,"{""type"": ""Point"", ""coordinates"": [-123.0352463..."


In [61]:
gardenDF.describe()

,STREET_NUMBER,NUMBER_OF_PLOTS
count,163.000000,167.000000
mean,2496.840491,30.000000
std,2166.223946,50.376295
min,1.000000,0.000000
25%,857.000000,5.000000
50%,2021.000000,12.000000
75%,3349.000000,38.000000
max,8725.000000,507.000000


So we can tell things like the gardens were created (`YEAR_CREATED`), and where the are (`LATITUDE`, `LONGITUDE`), and who's responsible for them (`STEWARD_OR_MANAGING_ORGANIZATION`). 

Now it is time to clean the data. This is a hugely important step and will generally eat a lot of your time, but it is worth doing right. Having mistakes in your data undermine everything you are trying to do.

First let's look at the index, the default is to index by integer, but we could have picked any column instead. It looks like the first column is unique (`MAPID`) and so let's use that (chosing the index right can make your life much easier when adding data or combining multiple DataFrames). The `inplace=True` argument means modify the existing dataframe rather than returning a modified copy

In [62]:
gardenDF.set_index('MAPID', inplace=True)

In [63]:
gardenDF.head(2)

,YEAR_CREATED,NAME,STREET_NUMBER,STREET_DIRECTION,STREET_NAME,STREET_TYPE,MERGED_ADDRESS,NUMBER_OF_PLOTS,NUMBER_OF_FOOD_TREES,NOTES,FOOD_TREE_VARIETIES,OTHER_FOOD_ASSETS,JURISDICTION,STEWARD_OR_MANAGING_ORGANIZATION,PUBLIC_E_MAIL,WEBSITE,Geo Local Area,Geom
MAPID,,,,,,,,,,,,,,,,,,
FA004,2007,1755 West 14th - Private apartment building ga...,1755.0,W,14th,Av,"1755 W 14th Av, Vancouver, BC",NaN,NaN,NaN,NaN,NaN,Private,Apartment residents,NaN,NaN,Fairview,"{""type"": ""Point"", ""coordinates"": [-123.1448703..."
FA010,NaN,Barclay Heritage Square,1433.0,NaN,Barclay,St,"1433 Barclay St, Vancouver, BC",0.0,2,NaN,Apple,NaN,Park Board,NaN,NaN,NaN,West End,"{""type"": ""Point"", ""coordinates"": [-123.1328450..."


One useful trick when cleaning data is to look at the unique values in a column. You'll often catch coding mistakes or values being used as placeholders this way, e.g.

In [64]:
gardenDF['YEAR_CREATED'].unique()

array(['2007', nan, '2011', '2013', '2008', '2009', '2010', '1993',
       '2006', '1992', '1995', '2014', '2012', 'pre 2000', '2015', '1986',
       '2003', '1996', '1988', '1990', '1985', '1999', '2016', '2017',
       '2018', '2019', 'Pre-2010', 'pre-1970', '1942', '1978', '2002'],
      dtype=object)

`Pre-2010`, `pre-1970` and `pre 2000` are kind of useless (and inconsistent!) so let's toss them

In [65]:
for badLabel in ['Pre-2010', 'pre-1970', 'pre 2000']:
    gardenDF = gardenDF[gardenDF['YEAR_CREATED'] != badLabel]

gardenDF['YEAR_CREATED'].unique()

array(['2007', nan, '2011', '2013', '2008', '2009', '2010', '1993',
       '2006', '1992', '1995', '2014', '2012', '2015', '1986', '2003',
       '1996', '1988', '1990', '1985', '1999', '2016', '2017', '2018',
       '2019', '1942', '1978', '2002'], dtype=object)

In [66]:
gardenDF.shape

(168, 18)

There's still a `nan`, and the years are strings (numbers would be better, or even dates). Pandas is pretty smart about dealing with missing data, but if that isn't enough there are methods like `dropna()` which will tell pandas to remove them from results or you can remove the problem entries with fancy indexing. For exampe, if we try to convert the `YEAR_CREATED` column to an integer blindly, it will barf on the `nan` (`NaN` is defined for floats but not ints, blame the IEEE)

In [67]:
#gardenDF['YEAR_CREATED'].astype(int)

But we can use the `.notnull` method to figure out where those rows are. This will give us a boolean array which we can use for Fancy Indexing

In [68]:
gardenDF['YEAR_CREATED'].notnull()

MAPID
FA004     True
FA010    False
FA012     True
FA016     True
FA018     True
         ...  
FA167     True
FA168     True
FA170     True
FA173     True
FA179     True
Name: YEAR_CREATED, Length: 168, dtype: bool

In [69]:
print('before',gardenDF.shape)
gardenDF = gardenDF[gardenDF['YEAR_CREATED'].notnull()]
print('after',gardenDF.shape)
gardenDF.head(2)


before (168, 18)
after (155, 18)


,YEAR_CREATED,NAME,STREET_NUMBER,STREET_DIRECTION,STREET_NAME,STREET_TYPE,MERGED_ADDRESS,NUMBER_OF_PLOTS,NUMBER_OF_FOOD_TREES,NOTES,FOOD_TREE_VARIETIES,OTHER_FOOD_ASSETS,JURISDICTION,STEWARD_OR_MANAGING_ORGANIZATION,PUBLIC_E_MAIL,WEBSITE,Geo Local Area,Geom
MAPID,,,,,,,,,,,,,,,,,,
FA004,2007,1755 West 14th - Private apartment building ga...,1755.0,W,14th,Av,"1755 W 14th Av, Vancouver, BC",NaN,NaN,NaN,NaN,NaN,Private,Apartment residents,NaN,NaN,Fairview,"{""type"": ""Point"", ""coordinates"": [-123.1448703..."
FA012,2011,Bosman Hotel,1060.0,NaN,Howe,St,"1060 Howe St, Vancouver, BC",12.0,NaN,NaN,NaN,NaN,Private,Bosman Hotel Society,NaN,NaN,Downtown,"{""type"": ""Point"", ""coordinates"": [-123.1250943..."


Now we could make `YEAR_CREATED` an integer (or date, see later).

In [70]:
gardenDF['YEAR_CREATED'] = gardenDF['YEAR_CREATED'].astype(int)

In [71]:
gardenDF['YEAR_CREATED'].describe()

count     155.000000
mean     2009.303226
std         9.212762
min      1942.000000
25%      2008.000000
50%      2011.000000
75%      2014.000000
max      2019.000000
Name: YEAR_CREATED, dtype: float64

The column dtype *is* int64, but `describe()` wants floats to work with so it gets converted.

Doing these steps every time for each column can be a lot of work and code, fortunately `read_csv` (and the other ingestion methods) can do most of the work while we're reading in the data. 

  * **delimiter=**: Sometimes a csv is a tsv, tabs are evil
  * **names=**: Pass a list of names to use for the columns
  * **usecols=**: Only slurp up a subset of columns
  * **skiprows=**: Ignore a number of rows at the top of the file
  * **na_values=**: Flag values which the CSV author used to indicate missing data, e.g. -1
  * **encoding=**: ...
  * **converters=**: Do some transformation on the columns before importing them
  * **parse_dates**=: Turn strings into dates
 
Some of these are obvious (e.g. pass in a list of names) but if you look at the documentation they are *really* flexible. Many will accept combinations of columns, functions, dictionaries of functions, etc. For dates you might have years in one column, months in another and days in a third. `parse_date` can combine these into a single `DateTime` object. Other times you might want to transform a column (e.g. convert to int or subtract off a constant), `converters` lets you specify functions to be run on columns during the ingestion.



In [72]:
!head PandasIntroSolved.ipynb

{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Pandas\n",
    "**Ian Allison, Compute Canada, 2020** (with several changes and additions for GOPH699)\n",
    "\n",
    "Pandas (Python Data Analysis Library) is a swiss-army knife module that you'll find at the top of a huge proportion of notebooks. It's popular enough to deserve its own import idiom..."


JSON (JavaScript Object Notation is an open standard file format and data interchange format that uses human-readable text to store and transmit data objects consisting of attribute–value pairs and arrays (or other serializable values). It is a very common data format, with a diverse range of applications, one example being web applications that communicate with a server. JSON is a language-independent data format. It was derived from JavaScript, but many modern programming languages include code to generate and parse JSON-format data. JSON filenames use the extension ".json". (wiki)

JSON is very nearly valid Python code with basic types being objects (dicts), arrays (lists), strings, numbers, booleans and nulls.

In [73]:
import json

geom = gardenDF.loc['FA013']['Geom']
json.loads(geom)['coordinates']

[-123.09573323, 49.26416489]

In [74]:
import json

def geom2lat(geom):
    if geom:
        return json.loads(geom)['coordinates'][0]
    else:
        return np.NaN


gardenDF = pd.read_csv(
    "CommunityGardensAndFoodTrees.csv",
    usecols = [
        'MAPID',
        'YEAR_CREATED',
        'NAME',
        'STEWARD_OR_MANAGING_ORGANIZATION',
        'STREET_NUMBER',
        'STREET_NAME',
        'Geom',
        'Geom'
    ],
    delimiter=';',
    encoding='latin1',
    na_values={
        'YEAR_CREATED': ['Pre-2010', 'pre-1970', 'pre 2000', 'nan']
    },
    index_col='MAPID',
    parse_dates=['YEAR_CREATED'],
    converters={
        'Geom': geom2lat,
    }
)
gardenDF.head()

,YEAR_CREATED,NAME,STREET_NUMBER,STREET_NAME,STEWARD_OR_MANAGING_ORGANIZATION,Geom
MAPID,,,,,,
FA004,2007-01-01,1755 West 14th - Private apartment building ga...,1755.0,14th,Apartment residents,-123.144870
FA010,NaT,Barclay Heritage Square,1433.0,Barclay,NaN,-123.132845
FA012,2011-01-01,Bosman Hotel,1060.0,Howe,Bosman Hotel Society,-123.125094
FA016,2013-01-01,Cambie Park Community Garden,500.0,54th,Marpole Oakridge Community Association,-123.118628
FA018,2011-01-01,Cartier Park,390.0,36th,NaN,-123.097511


### Same thing but with JSON/API

[opendata vancouver](https://opendata.vancouver.ca/api/v2/console) publishes the same datasets as an API. The requests module lets you talk to that API and gives you results in JSON to play with. `pandas` can then consume that JSON, e.g.

In [75]:
import requests

base_url = 'https://opendata.vancouver.ca/api/records/1.0/search/'
headers = {
 'Content-Type': 'application/json; charset=utf-8'
}


params = {
    'dataset' : 'community-gardens-and-food-trees',
    'q'       : '',
    'rows'    : 10,
    'facets'  : ['year_created', 'juristiction', 'geo_local_area']
}


r = requests.get(base_url, params=params)

In [76]:
r.json().keys()

dict_keys(['nhits', 'parameters', 'records'])

In [77]:
r.json()['records'][0]['fields']

{'name': '15th Avenue Coop',
 'street_number': '1255',
 'jurisdiction': 'Private',
 'geom': {'coordinates': [-123.0788387, 49.2571193], 'type': 'Point'},
 'street_name': '15th',
 'year_created': '2014',
 'street_type': 'Av',
 'street_direction': 'E',
 'merged_address': '1255 E 15th Av, Vancouver, BC',
 'geo_local_area': 'Mount Pleasant',
 'mapid': 'FA002',
 'number_of_plots': 8}

So we can do something like

### Grouping, Joining, Concatenating

You can get pretty far by jamming everything into a single dataframe, but sometimes you might want to do aggregate operations within a dataframe (e.g. group together all of the rows by year and show the mean value of some other column). Alternatively you might want to add new rows to or columns to an existing DataFrame or join dataframes based on shared key.


In [78]:
carsDF = pd.read_csv(
    'https://raw.githubusercontent.com/mwaskom/seaborn-data/master/mpg.csv'
)
carsDF.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name
0,18.0,8,307.0,130.0,3504,12.0,70,usa,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,usa,buick skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,usa,plymouth satellite
3,16.0,8,304.0,150.0,3433,12.0,70,usa,amc rebel sst
4,17.0,8,302.0,140.0,3449,10.5,70,usa,ford torino


### Groupby
Let's group things by number_of_cylinders and see how that affects mpg...

In [79]:
carsDF['mpg'].mean()

23.514572864321607

Calling groupby on it's own will give you a `DataFrameGroupBy` object, you have to tell it what you want to do with the groups to actually see some results, this can be convenient if you want to look at different aggregate functions on the same groups.

In [80]:
carsDFbyCylinders = carsDF.groupby('cylinders')
carsDFbyCylinders

The typical aggregate operations are things like

  * mean()
  * sum()
  * median()
  * min()/max()

In [81]:
carsDFbyCylinders.mean()

,mpg,displacement,horsepower,weight,acceleration,model_year
cylinders,,,,,,
3,20.550000,72.500000,99.250000,2398.500000,13.250000,75.500000
4,29.286765,109.796569,78.281407,2308.127451,16.601471,77.073529
5,27.366667,145.000000,82.333333,3103.333333,18.633333,79.000000
6,19.985714,218.142857,101.506024,3198.226190,16.263095,75.928571
8,14.963107,345.009709,158.300971,4114.718447,12.955340,73.902913


Group the carsDF by model_year and look at the median mpg (don't include the other columns)

In [82]:
carsDF.groupby('model_year')['mpg'].median()

model_year
70    16.00
71    19.00
72    18.50
73    16.00
74    24.00
75    19.50
76    21.00
77    21.75
78    20.70
79    23.90
80    32.70
81    31.60
82    32.00
Name: mpg, dtype: float64

You can actually do much more with groupby, you can iterate over the groups

In [83]:
for vroom, group in carsDFbyCylinders:
    print(f"There are {group.shape[0]} cars with {vroom} cylinders")

There are 4 cars with 3 cylinders
There are 204 cars with 4 cylinders
There are 3 cars with 5 cylinders
There are 84 cars with 6 cylinders
There are 103 cars with 8 cylinders


How are there possibly 4 cars with 3 cylinders?!

We can apply multiple operations at the same time. The `.aggregate()` method can take a list of the operations you want to perform (e.g. ["max", "min"])

In [84]:
carsDFbyCylinders['mpg'].aggregate(["min","max"])

,min,max
cylinders,,
3,18.0,23.7
4,18.0,46.6
5,20.3,36.4
6,15.0,38.0
8,9.0,26.6


We can filter based on group, this is a silly example, but group the cars by cylinder, then show me all of the groups with a mean mpg > 15.

In [85]:
carsDFbyCylinders.filter(lambda x: x['mpg'].mean()>15).head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name
14,24.0,4,113.0,95.0,2372,15.0,70,japan,toyota corona mark ii
15,22.0,6,198.0,95.0,2833,15.5,70,usa,plymouth duster
16,18.0,6,199.0,97.0,2774,15.5,70,usa,amc hornet
17,21.0,6,200.0,85.0,2587,16.0,70,usa,ford maverick
18,27.0,4,97.0,88.0,2130,14.5,70,japan,datsun pl510


The transform method lets you perform a group operation then use the results to update the rows. For example, we could calculate mean values for our groups, then look at how individual cars perform relative to that

In [86]:
carsDFbyCylinders.transform(lambda x: x - x.mean()).head()

/tmp/ipykernel_21290/1432915777.py:1: FutureWarning: Dropping invalid columns in DataFrameGroupBy.transform is deprecated. In a future version, a TypeError will be raised. Before calling .transform, select only columns which should be valid for the transforming function.
  carsDFbyCylinders.transform(lambda x: x - x.mean()).head()


,mpg,displacement,horsepower,weight,acceleration,model_year
0,3.036893,-38.009709,-28.300971,-610.718447,-0.95534,-3.902913
1,0.036893,4.990291,6.699029,-421.718447,-1.45534,-3.902913
2,3.036893,-27.009709,-8.300971,-678.718447,-1.95534,-3.902913
3,1.036893,-41.009709,-8.300971,-681.718447,-0.95534,-3.902913
4,2.036893,-43.009709,-18.300971,-665.718447,-2.45534,-3.902913


There is also an apply method which is even more general and will let you apply an arbitrary function to the group results.